## Group 7
##### Alec Arreola-Coburn, Cici Liu, Philip McCabe, Rishabh Shyamsunder, Elena Wu-Ratner

***

### Chapter 5, Slide 7 - The BMC Transshipment Problem

    Supply 500 > Demand 480, Inflow - Outflow >= Demand/Supply
   
    Decision variables: Xij for # of shipments from i to j
    Objective Function: MIN 30X12 + 40X14 + 50X23 + ...
                      : MIN SUM(Cij x Xij)
    S.T:
        Newark: X12 + X14 >= 200
        Boston: X12 - X 23 >= 100
        Columbus: X23 + X 53 - X35 >= 60
        Richmond: X14 + X74 + X54 >= 80
        Atlanta: X75 + X65 + X35 - (X53 + X56 + X54) >= 170
        Mobile: X56 + X76 - X65 >= 70
        J'ville: X76 + X75 + X74 >= 300

In [1]:
from gurobipy import *
#node_names = ['Newark','Boston','Columbus','Richmond','Atlanta','Mobile',"J'ville"]
#origin = ['Newark',"J'ville"]
#destination = ['Boston','Columbus','Richmond','Atlanta','Mobile']

In [2]:
"""paths, cost = multidict(
{('Newark','Boston'): 30,
('Newark','Richmond'):40,
('Boston','Columbus'):50,
('Columbus','Atlanta'):35,
('Atlanta','Columbus'):40,
('Atlanta','Richmond'):30,
('Atlanta','Mobile'):35,
('Mobile','Atlanta'):25,
("J'ville",'Richmond'):50,
("J'ville",'Atlanta'):45,
("J'ville",'Mobile'):50})

flow = {
    ('Newark'):-200,
    ("J'ville"):-300,
    ('Boston'):100,
    ('Columbus'):60,
    ("Richmond"):80,
    ('Atlanta'):170,
    ('Mobile'):70,
}

capacity = {
('Newark','Boston'): 100,
('Newark','Richmond'):120,
('Boston','Columbus'):160,
('Columbus','Atlanta'):230,
('Atlanta','Columbus'):230,
('Atlanta','Richmond'):250,
('Atlanta','Mobile'):240,
('Mobile','Atlanta'):240,
("J'ville",'Richmond'):220,
("J'ville",'Atlanta'):130,
("J'ville",'Mobile'):230}"""

'paths, cost = multidict(\n{(\'Newark\',\'Boston\'): 30,\n(\'Newark\',\'Richmond\'):40,\n(\'Boston\',\'Columbus\'):50,\n(\'Columbus\',\'Atlanta\'):35,\n(\'Atlanta\',\'Columbus\'):40,\n(\'Atlanta\',\'Richmond\'):30,\n(\'Atlanta\',\'Mobile\'):35,\n(\'Mobile\',\'Atlanta\'):25,\n("J\'ville",\'Richmond\'):50,\n("J\'ville",\'Atlanta\'):45,\n("J\'ville",\'Mobile\'):50})\n\nflow = {\n    (\'Newark\'):-200,\n    ("J\'ville"):-300,\n    (\'Boston\'):100,\n    (\'Columbus\'):60,\n    ("Richmond"):80,\n    (\'Atlanta\'):170,\n    (\'Mobile\'):70,\n}\n\ncapacity = {\n(\'Newark\',\'Boston\'): 100,\n(\'Newark\',\'Richmond\'):120,\n(\'Boston\',\'Columbus\'):160,\n(\'Columbus\',\'Atlanta\'):230,\n(\'Atlanta\',\'Columbus\'):230,\n(\'Atlanta\',\'Richmond\'):250,\n(\'Atlanta\',\'Mobile\'):240,\n(\'Mobile\',\'Atlanta\'):240,\n("J\'ville",\'Richmond\'):220,\n("J\'ville",\'Atlanta\'):130,\n("J\'ville",\'Mobile\'):230}'

In [2]:
nodes = 7
origin = 1
destination = 7
#origin = [1,7]
#destination = [2,3,4,5,6]

paths, cost = multidict(
{(1,2):30,
(1,4):40,
(2,3):50,
(3,5):35,
(5,3):40,
(5,4):30,
(5,6):35,
(6,5):25,
(7,6):50,
(7,5):45,
(7,4):50})

capacity = {
    (1):200,
    (7):300,
    (2):100,
    (3):60,
    (4):80,
    (5):170,
    (6):70,
}

In [3]:
t = Model("Transshipment")

GurobiError: License expired 2020-10-14 - license file '/Users/liucathy/gurobi.lic'

In [5]:
cost

{(1, 2): 30,
 (1, 4): 40,
 (2, 3): 50,
 (3, 5): 35,
 (5, 3): 40,
 (5, 4): 30,
 (5, 6): 35,
 (6, 5): 25,
 (7, 6): 50,
 (7, 5): 45,
 (7, 4): 50}

In [6]:
route = t.addVars(paths, obj=cost, name='route')

In [7]:
for i in range(1,nodes+1):
    t.addConstr(sum(route[i,j] for i,j in path.select(i,'*'))
               -sum(route[j,i] for j,i in path.select('*',i))
               == (1 if i == origin else -1 if i == destination else 0))

TypeError: cannot unpack non-iterable Var object

In [4]:
t.optimize()

NameError: name 't' is not defined

In [ ]:
print('The path with the least cost is:')
for i,j in path:
    if(route[i,j].x > 0):
        print(i, j, ":", route[i,j].x)
print('Total Cost: ' + str(t.objVal))
print()

***

### Chapter 5, Slide 17 - Coal Bank Hollow Recycling Problem

    Supply 190 > Demand 150, Inflow - Outflow >= Demand/Supply
    
    Decision Varaibles: Xij
    Objective Function: MIN 13X15 + 12X16 + 11X25 + 13X26...
                      : MIN SUM(Cij x Xij)
    S.T:
        Node 1: X15 + X16 >= 70
        Node 2: X25 + X26 >= 50
        Node 3: X35 + X36 >= 30
        Node 4: X45 + X46 >= 40
        Node 5: 0.9X15 + 0.8X25 + 0.95X35 + 0.75X45 – (X57 + X58 + X59) >= 0
        Node 6: 0.85X16 + 0.85X26 _ 0.9X36 + 0.85X46 - (X67 + X68 + X69) >= 0
        Node 7: 0.95X57 + 0.9X67 >= 60
        Node 8: 0.9X58 + 0.95X68 >= 40
        Node 9: 0.9X59 + 0.95X69 >= 50

In [18]:
paper = ['Newspaper','Mixed paper','White office paper','Cardboard']
nodes = ['1','2','3','4','5','6']

arcs, capacity = multidict({
    ('1','5'):0.9,
    ('1','6'):0.85,
    ('2','5'):0.8,
    ('2','6'):0.85,
    ('3','5'):0.95,
    ('3','6'):0.9,
    ('4','5'):0.75,
    ('4','6'):0.85
})

cost = {
    ('Newspaper','1','5'): 13,
    ('Newspaper','1','6'): 12,
    ('Mixed paper','2','5'): 11,
    ('Mixed paper','2','6'): 13,
    ('White office paper','3','5'): 9,
    ('White office paper','3','6'): 10,
    ('Cardboard','4','5'): 13,
    ('Cardboard','4','6'): 14
}

inflow = {
    ('Newspaper','1'): 70,
    ('Newspaper','5'): 0,
    ('Newspaper','6'): 0,
    ('Mixed paper','2'): 50,
    ('Mixed paper','5'): 0,
    ('Mixed paper','6'): 0,
    ('White office paper','3'): 30,
    ('White office paper','5'): 0,
    ('White office paper','6'): 0,
    ('Cardboard','4'): 40,
    ('Cardboard','5'): 0,
    ('Cardboard','6'): 0
}

In [19]:
processes = ['Process 1','Process 2']
nodes2 = ['5','6','7','8','9']

arcs2, capacity2 = multidict({
    ('5','7'):0.95,
    ('5','8'):0.9,
    ('5','9'):0.9,
    ('6','7'):0.9,
    ('6','8'):0.95,
    ('6','9'):0.95
})

cost2 = {
    ('Process 1','5','7'): 5,
    ('Process 1','5','8'): 6,
    ('Process 1','5','9'): 8,
    ('Process 2','6','7'): 6,
    ('Process 2','6','8'): 8,
    ('Process 2','6','9'): 7,
}

inflow2 = {
    ('Process 1','5'):0,
    ('Process 1','7'):60,
    ('Process 1','8'):40,
    ('Process 1','9'):50,
    ('Process 2','6'):0,
    ('Process 2','7'):60,
    ('Process 2','8'):40,
    ('Process 2','9'):50,
}

In [20]:
r = Model('Recycling')

In [21]:
flow = r.addVars(paper, arcs, obj=cost, name='flow')

In [22]:
flow

{('Newspaper', '1', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '1', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '2', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '2', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '3', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '3', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '4', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Newspaper', '4', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '1', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '1', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '2', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '2', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '3', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '3', '6'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '4', '5'): <gurobi.Var *Awaiting Model Update*>,
 ('Mixed paper', '4', '6')

In [23]:
# Capacity constraints..?
r.addConstrs(
    (flow.sum('*',i,j) <= capacity[i,j] for i,j in arcs),'cap')

{('1', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('1', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('2', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('2', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('3', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('3', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('4', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('4', '6'): <gurobi.Constr *Awaiting Model Update*>}

In [24]:
# Flow conservation constraints...?
r.addConstrs(
    (flow.sum(h,'*',j) + inflow[h,j] == flow.sum(h,j,'*')
    for h in paper for j in nodes[4:]), 'node')

{('Newspaper', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('Newspaper', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('Mixed paper', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('Mixed paper', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('White office paper', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('White office paper', '6'): <gurobi.Constr *Awaiting Model Update*>,
 ('Cardboard', '5'): <gurobi.Constr *Awaiting Model Update*>,
 ('Cardboard', '6'): <gurobi.Constr *Awaiting Model Update*>}

In [25]:
flow2 = r.addVars(processes, arcs2, obj=cost2, name='flow2')

In [26]:
r.addConstrs(
    (flow2.sum('*',i,j) <= capacity2[i,j] for i,j in arcs2), 'cap2')

{('5', '7'): <gurobi.Constr *Awaiting Model Update*>,
 ('5', '8'): <gurobi.Constr *Awaiting Model Update*>,
 ('5', '9'): <gurobi.Constr *Awaiting Model Update*>,
 ('6', '7'): <gurobi.Constr *Awaiting Model Update*>,
 ('6', '8'): <gurobi.Constr *Awaiting Model Update*>,
 ('6', '9'): <gurobi.Constr *Awaiting Model Update*>}

In [27]:
r.addConstrs(
    (flow2.sum(h,'*',j) + inflow2[h,j] == flow2.sum(h,j,'*')
    for h in processes for j in nodes2[2:]), 'node2')

{('Process 1', '7'): <gurobi.Constr *Awaiting Model Update*>,
 ('Process 1', '8'): <gurobi.Constr *Awaiting Model Update*>,
 ('Process 1', '9'): <gurobi.Constr *Awaiting Model Update*>,
 ('Process 2', '7'): <gurobi.Constr *Awaiting Model Update*>,
 ('Process 2', '8'): <gurobi.Constr *Awaiting Model Update*>,
 ('Process 2', '9'): <gurobi.Constr *Awaiting Model Update*>}

In [28]:
r.update()

In [29]:
r.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 28 rows, 44 columns and 88 nonzeros
Model fingerprint: 0x0d9139a1
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 6e+01]
Presolve removed 16 rows and 32 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds
Infeasible model
